In [1]:
import numpy as np
import pickle
import tensorflow as tf

In [3]:
#pickle file 업로드 필요
#with open("/content/drive/MyDrive/Colab Notebooks/deep_learning_start/sample_weight.pkl", 'rb') as f: #저장된 wieght파일을 불러와 출력
with open("/content/drive/MyDrive/Colab Notebooks/sample_weight.pkl", 'rb') as f: #저장된 wieght파일을 불러와 출력
    data = pickle.load(f) #binary 형태로 저장되어 있다가 불러오면서 저장당시 class로 복원
#print(data)
print(f"W1: {data['W1']}") #여기서는 dictionary 형태
print(f"b1: {data['b1']}") # 가중치와 bias 값을 초기화

W1: [[-0.00741249 -0.00790439 -0.01307499 ...  0.01978721 -0.04331266
  -0.01350104]
 [-0.01029745 -0.01616653 -0.01228376 ...  0.01920228  0.02809811
   0.01450908]
 [-0.01309184 -0.00244747 -0.0177224  ...  0.00944778  0.01387301
   0.03393568]
 ...
 [ 0.02242565 -0.0296145  -0.06326169 ... -0.01012643  0.01120969
   0.01027199]
 [-0.00761533  0.02028973 -0.01498873 ...  0.02735376 -0.01229855
   0.02407041]
 [ 0.00027915 -0.06848375  0.00911191 ... -0.03183098  0.00743086
  -0.04021148]]
b1: [-0.06750315  0.0695926  -0.02730473  0.02256093 -0.22001474 -0.22038847
  0.04862635  0.13499236  0.23342554 -0.0487357   0.10170191 -0.03076038
  0.15482435  0.05212503  0.06017235 -0.03364862 -0.11218343 -0.26460695
 -0.03323386  0.13610415  0.06354368  0.04679805 -0.01621654 -0.05775835
 -0.03108677  0.10366164 -0.0845938   0.11665157  0.21852103  0.04437255
  0.03378392 -0.01720384 -0.07383765  0.16152057 -0.10621249 -0.01646949
  0.00913961  0.10238428  0.00916639 -0.0564299  -0.10607515  

In [4]:
def sigmoid(x):
  return 1/(1+np.exp(-x))

In [5]:
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

# overflow 방지
def softmax(a):
    c = np.max(a) # list a 중 가장 큰 값
    exp_a = np.exp(a-c)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

In [6]:
def get_data():
    #(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, flatten=True, one_hot_label=False)
    (x_train,t_train),(x_test,t_test) = tf.keras.datasets.mnist.load_data()
    x_test = x_test.reshape(10000,-1)
    x_test = x_test.astype('float32') / 255. #normalize 데이터를 0과 1사이로
    return x_test, t_test

def init_network():
    with open("/content/drive/MyDrive/Colab Notebooks/sample_weight.pkl", 'rb') as f: #저장된 wieght파일을 불러와 model에 적용
        network = pickle.load(f)
    return network


def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3'] #위와 같은 3층 layer 구현
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1     #(10000, 784)(784, 50)  => (10000, 50)
    z1 = sigmoid(a1)
    a2 = np.dot(z1, W2) + b2    # (10000, 50)(50, 100)  => (10000, 100)
    z2 = sigmoid(a2)
    a3 = np.dot(z2, W3) + b3    # (10000, 100)(100, 10) => (60000,10)
    y = softmax(a3)
    return y

x, t = get_data()
network = init_network() # pickle data를 가져와(parameter 값) 네트워크에 세팅 
#print(W1.shape)
#print(W2.shape)
#print(W3.shape)

batch_size = 100
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    x_batch = x[i:i+batch_size] # 이미지 1개씩이 아닌 batch size만큼의 이미지를 넘겨줘 한번에 연산
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    accuracy_cnt += np.sum(p == t[i:i+batch_size])

print("Accuracy:" + str(float(accuracy_cnt) / len(x)))
print(x[0:0+batch_size].shape)

11501568/11490434 [==============================] - 0s 0us/step
Accuracy:0.9352
(100, 784)
